### ONNX Model

In [19]:
!pip3 install onnxruntime
!pip3 install onnxmltools

  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
     |████████████████████████████████| 302 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 12.8 MB 102.9 MB/s eta 0:00:01
     |████████████████████████████████| 278 kB 106.1 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 1.2 MB/s s eta 0:00:01


In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.mllib.stat import Statistics
from pyspark.ml.linalg import DenseVector
from pyspark.sql import functions as F

In [23]:
import random, os
import numpy as np
from pyspark.sql import Row
from sklearn import neighbors
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.stat import Statistics
from pyspark.ml import PipelineModel

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
from pyspark.sql import SparkSession

In [39]:
import onnxruntime
import onnxmltools

In [27]:
spark = SparkSession\
    .builder\
    .appName("ONNX_Model")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
    .config("spark.yarn.access.hadoopFileSystems",os.environ["STORAGE"])\
    .getOrCreate()

In [55]:
def load_pipeline():
    modelPipeline = PipelineModel.load(os.environ["STORAGE"]+"/pdefusco/pipeline")
    return modelPipeline

In [56]:
modelPipeline = load_pipeline()

In [29]:
df = spark.sql("SELECT * FROM default.lc_smote_subset LIMIT 10")

Hive Session ID = fc9e33dc-ad5d-4f9f-bc80-0d57d2e86059


In [59]:
def transform_pipeline(spark_df, pmodel):        
     
    cols = ['acc_now_delinq', 'acc_open_past_24mths', 'annual_inc', 'avg_cur_bal', 'funded_amnt']
        
    spark_df = spark_df.select(*cols)
    
    for c in spark_df.columns:
        spark_df = spark_df.withColumn(c, spark_df[c].cast("float"))
        
    return pmodel.transform(spark_df)

In [60]:
sample_df = transform_pipeline(df, modelPipeline)

In [65]:
sample_df = sample_df.select(['acc_now_delinq', 'acc_open_past_24mths', 'annual_inc', 'avg_cur_bal', 'funded_amnt'])

In [70]:
initial_types = onnxmltools.convert.sparkml.buildInitialTypesSimple(sample_df)

In [74]:
initial_types

[('acc_now_delinq', FloatTensorType(shape=[1, 1])),
 ('acc_open_past_24mths', FloatTensorType(shape=[1, 1])),
 ('annual_inc', FloatTensorType(shape=[1, 1])),
 ('avg_cur_bal', FloatTensorType(shape=[1, 1])),
 ('funded_amnt', FloatTensorType(shape=[1, 1]))]

In [75]:
initial_types = [
    ("label", StringTensorType([1, 1])),
    # (repeat for the required inputs)
]

NameError: name 'StringTensorType' is not defined

In [73]:
onnx_model = onnxmltools.convert.convert_sparkml(modelPipeline, 'My Sparkml Pipeline', initial_types)

TypeError: value "[-0.021346803970621823,-0.020154494863575963,-0.03151374209034207,0.05956225586563654,0.12421686806051492]" is not valid attribute data type.

In [72]:
onnxmltools.convert.convert_sparkml()

TypeError: convert_sparkml() missing 1 required positional argument: 'model'

In [21]:
spark.stop()